# 写真の減色

画像データは1600万色程度の色彩を表現できるが、これは一般的な写真の画素数よりも多い。つまり、全く使っていない色はたくさんある。また、次の写真のように、カラー写真とはいっても、実際に使っている色はそれほど多くないように見えるものもある。

![](https://live.staticflickr.com/8380/8640855620_102dda223f_z_d.jpg)

(CC BY 2.0 2009 SteFou! via Flickr)

この写真を、できるだけ少ない色数で表示してみよう。

## 白黒の場合

まず、この画像を入手し、色彩を落として、numpy arrayの形にする。414 x 640W pixelの画像なので、サイズ(414,640)の実数のarrayの各要素がそれぞれの画素の明るさを表す。


In [ ]:
from imageio import imread
import PIL

img = imread("https://live.staticflickr.com/8380/8640855620_102dda223f_z_d.jpg")
img.shape

画像の画素ごとのデータは、通常はR(赤),G(緑),B(青)それぞれ0〜255の256段階で表現される。3色を平均して白黒画像にする。

In [ ]:
# RGB方向の平均をとり、255で割る。
import numpy as np

#平均値は実数になってしまうので、8ビット負号なし整数(0〜255)に変換しておく。
gray = np.average(img, axis=2).astype(np.uint8)
gray.shape

In [ ]:
gray

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

# displayはJupyterの機能。
display(PIL.Image.fromarray(gray))

## 単純に階調を8等分して、8階調にする

何も考えず、この画像を8階調に落してみます。0〜255の階調を8階調にするには、
* 0〜31を全部0に
* 32〜63を全部32に
* ...
* 224〜255を全部224に
置きかえればいいのです。

これには`floor()`関数が便利です。`floor(x)`関数は、`x`の小数点以下を切りすてて、`x`より小さい最大の整数を返します。

In [ ]:
simple8 = (np.floor(gray/32)*32).astype(np.uint8)
display(PIL.Image.fromarray(simple8))

なんかのっぺりとしてわかりにくい絵になってしまいました。コントラストが小さいせいでしょう。灰色の階調をもう少しこまかくとりたいところです。

## ヒストグラムにもとづいた彩色

そこで、ヒストグラムをまず作ります。

In [ ]:
hist = np.histogram(gray, bins=256)
hist

`np.histogram()`は2つのarrayを返します。1つめがヒストグラム、2つめはビン(区間)の目盛です。

In [ ]:
hist[0].shape, hist[1].shape

In [ ]:
plt.plot(hist[0])

予想通り、暗い色が少なく、明るい色に偏っていることがわかりました、暗い色の点は少ないので、明るいところを細かい階調で表現するのがよさそうです。

そこで、8階調に落とす時に、各階調の画素の数が均等になるようにしましょう。つまり、上のグラフを、面積が等しくなるように8等分します。

In [ ]:
height, width = gray.shape
Npix = height*width
# 1次元にして、輝度の小さい順にソートする。
pixels = np.sort(gray.reshape(Npix))
pixels

In [ ]:
# (Npix/8) 個目の画素は?
pixels[Npix//8]

なので、輝度が108以下の画素は全部輝度を108/2=54にする。

Npix 番目から Npix/4 番目の画素は、

In [ ]:
pixels[Npix//4]

なので、108以上128以下の画素は全部輝度を(108+128)/2=118にする。

以下同様。これを8階調まとめてやってみよう。

In [ ]:
for i in range(8):
    # range
    smallest = Npix*i//8
    largest  = Npix*(i+1)//8 - 1
    # brightness of the pixels at the two ends
    Ps = pixels[smallest]
    Pl = pixels[largest]
    # average of the two
    Pm = (Ps+Pl)/2
    print(i,Ps,Pl,Pm)

うまいぐあいにできている。これを使って、grayの中身を書きかえる。

numpyのとてもトリッキーな書き方を使ってみよう。

In [ ]:
# make a copy of gray
equi8 = gray.copy()

cond = gray < 108
equi8[cond] = 54

cond = (108 <= gray ) & (gray < 128)
equi8[cond] = 118

cond = (128 <= gray ) & (gray < 141)
equi8[cond] = 135

cond = (141 <= gray ) & (gray < 154)
equi8[cond] = 148

cond = (154 <= gray ) & (gray < 170)
equi8[cond] = 162

cond = (170 <= gray ) & (gray < 186)
equi8[cond] = 178

cond = (186 <= gray ) & (gray < 208)
equi8[cond] = 197

cond = 208 <= gray
equi8[cond] = 230




In [ ]:
display(PIL.Image.fromarray(equi8))

ちょっとメリハリがついて、背景と桜が見分けられるようになった。

## カラーの場合

デジタルカラー画像は赤、緑、青それぞれの強度が256段階あり、1600万通りの色彩がありうる。3色をそれぞれ8段階にしたとしても、64色は必要になる。8色にまで落とすためには、3色をon/offの2段階にまで落とす必要があり、やる前からうまくいかないのは目に見えている。

In [ ]:
simple = img.copy()
# 画素の輝度が128より小さい点はすべて0にする。
simple[img<128]=0
# 128より大きい点はすべて255にする。
simple[img>=128]=255
display(PIL.Image.fromarray(simple))

酷い。

そこで、k-平均分類という機械学習の手法を使ってみる。k-平均分類は、機械学習において、多次元空間に散在する多数の点を、近いもの同士で集めて、クラスターにする手法である。

桜の写真の画素は、R,G,Bを3つの軸とする立方体の中の点で表される。上の写真には640x414=265000点の画素があり、それらは立方体の中で偏って分布している。おそらくピンクに相当する部分に多数の点が集中し、ほかに緑や黒に相当する領域に小さな集団を作っているはずだ。

そこで、この3次元空間内の点を、近いものどうしをつないでいくことで、領域分割する手法がk-平均法 (k-means classifier)である。

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_kmeans_digits_001.png)

2次元でのk-近傍分類器はこんな感じ。10グループに分けろ、と指定すると、10種類に分けてくれる。それぞれのクラスターの重心点で色を代表させれば、色数を減らせる。

k平均法は、機械学習ライブラリscikit-learnに含まれているKMeansを使う。

In [ ]:
# Pythonデータサイエンスハンドブック 5.11.1
from sklearn.cluster import KMeans

#画像をピクセル列に変換する(そうしないとk-meansが使えない)
pixels = img.reshape(Npix, 3)

# 8つの代表的な色をさがさせる。
kmeans = KMeans(n_clusters=8)
kmeans.fit(pixels)
kmeans.cluster_centers_

In [ ]:
# それぞれのピクセルに一番近い中心は何番か。
kmeans.predict(pixels)

In [ ]:
# ピクセルごとの色の変換表を作る

new_pixels = kmeans.cluster_centers_[kmeans.predict(pixels)]
new_pixels

In [ ]:
# new_pixelsを8ビット整数にし、arrayの形をもとに戻し、画像として表示する。
display(PIL.Image.fromarray(new_pixels.astype(np.uint8).reshape(height, width, 3)))

たった8色でもここまで雰囲気が出せました。